# TP4 Ciencia de datos - Redes

## 1. Creación del Grafo
A partir del archivo de datos, genere el grafo de colaboraciones entre autores. Cuente el número de nodos y aristas. Haga una representación gráfica del grafo.

In [1]:
import matplotlib as mpl
import numpy as np
import matplotlib.pyplot as plt
import csv
import networkx as nx
import seaborn
from random import choice
import operator

def get_autores(linea):
    autores = linea[3]
    car = '\/ .~-()`'
    for c in car:
        autores = autores.replace(c, '')
    return autores.upper().split('&')

def add_paper(G, linea):
    autores = get_autores(linea)
    
    for autor in autores:
        G.add_node(autor)
    
    e = 1
    for autor in autores:
        if e < len(autores):
            edges = zip([autor]*(len(autores)-e), autores[e:])
            G.add_edges_from(edges)
        e = e + 1

In [2]:
with open('data.csv', 'r') as data:
    papers = csv.reader(data)
    G = nx.Graph()
    for paper in papers:
        add_paper(G, paper)

cant_nodos = len(list(G.nodes()))
cant_aristas = len(list(G.edges()))
print("Cantidad de nodos: " + str(cant_nodos))
print("Cantidad de aristas: " + str(cant_aristas))

Cantidad de nodos: 3935
Cantidad de aristas: 7820


## 2. Distribución de Grado
Grafique la distribución del grado de los autores, en escalas lineales, semilogarítmica, y log-log. Argumente,a partir de los gráficos, de qué tipo de distribución se trata. 

In [19]:
dg = sorted([e[1] for e in list(nx.degree(G))], reverse=True)  

plt.plot(dg)
plt.title('Distribución de grado (lineal)')
plt.xlabel('Nodo')
plt.ylabel('Grado')
#plt.savefig('graficos/dg_lineal.jpg', dpi=300)
#plt.close()
plt.show()

plt.loglog(dg)
plt.title('Distribución de grado (loglog)')
plt.xlabel('Nodo')
plt.ylabel('Grado')
#plt.savefig('graficos/dg_loglog.jpg', dpi=300)
#plt.close()
plt.show()

plt.semilogx(dg)
plt.title('Distribución de grado (semilog x)')
plt.xlabel('Nodo')
plt.ylabel('Grado')
#plt.savefig('graficos/dg_semilogx.jpg', dpi=300)
#plt.close()
plt.show()

Se trata de una distribución exponencial, porque al ser representada en una gráfica semilog en x se comporta de forma aproximadamente lineal. 

## 3. Componentes Conexas
Calcule el número de componentes conexas del grafo. Muestre el tamaño de la componente mayor, o componente gigante.

In [24]:
ncc = nx.number_connected_components(G)
print("Numero de componentes conexas: " + str(ncc))
    
    # cg es la componente gigante
cg = max(nx.connected_component_subgraphs(G), key=len)
tamcg = len(list(cg.nodes()))
print("Tamaño de la componente gigante: " + str(tamcg))

Numero de componentes conexas: 513
Tamaño de la componente gigante: 3011


## 4. Tamaños de Vecindades
Trabajando con la componente gigante del grafo, estudie, parándose en un nodo al azar, cómo aumenta
el número de autores alcanzados a medida que se aleja del nodo semilla. Grafique el número de autores
alcanzados en función de la distancia al nodo semilla. Grafique también el número de nuevos autores
que se agregan en cada paso, y estime el máximo de esta función. Pruebe con varios nodos semillas y
analice la robustez de este resultado. Discuta el significado de los gráficos y su relación con el fenómeno
de seis grados de separación.

In [20]:
def tamañoVecino(cg):
    a=[]
    c=[]
    random_node = choice(list(cg.nodes()))
    for i in range (0,20):
        b=[]
        b=nx.ego_graph(cg, random_node, radius=i, center=False)
        if i==0:
            c.append(len(b))
        else:   
            c.append(len(b)-(a[i-1]))
 
        a.append(len(b))
    return a , c , c.index(max(c))

iterations=np.arange(1,10,1)
n= len(iterations)
colors = mpl.cm.gist_rainbow(np.linspace(0, 1, n))
maximos=[]

fig, ax = plt.subplots()
for color, i in zip(colors, iterations):
    ax.plot(tamañoVecino(cg)[0],color=color)
plt.title('Tamaño de vecindades')
plt.xlabel('Paso/distancia')
plt.ylabel('Numero de autores alcanzados')
#plt.savefig('graficos/tamañodeVecindades1.jpg', dpi=300)
#plt.close()
plt.show()

fig, ax1 = plt.subplots()
for color, i in zip(colors, iterations):
    ax1.plot(tamañoVecino(cg)[1],color=color)
    maximos.append(tamañoVecino(cg)[2])
plt.title('Autores nuevos por paso')
plt.xlabel('Paso')
plt.ylabel('Numero de autores nuevos')
#plt.savefig('graficos/tamañodeVecindades2.jpg', dpi=300)
#plt.close()
plt.show()

print('Paso asociado al mayor número de autores nuevos - valor medio de todas las iteraciones ' + str(np.mean(maximos)))

Paso asociado al mayor número de autores nuevos - valor medio de todas las iteraciones 6.88888888889


En ambos gráficos se evidencia que alrededor de los seis grados de separación existe una suerte de punto de inflexión en las curvas, a partir de ese valor el número de autores nuevos por paso empieza a decrecer. Por lo tanto, se evidencia que con 6 grados de separación se puede alcanzar a casi todos los nodos de la componente gigante. Esto correlaciona perfectamente con lo que se conoce como "fenómeno de los 6 grados de separación".    

## 5. Mundos Pequeños
Compute el coeficiente de clustering C y el camino mínimo medio l para la componente gigante. Genere
un grafo aleatorio con la misma distribución de grado y compute las mismas medidas para este grafo.
Compare e interprete los resultados. ¿Se trata de un grafo con estructura de mundos pequeños?

In [6]:
print('Grafo componente gigante')
nx.draw(cg)  # networkx draw()
plt.draw()  # pyplot draw()
plt.show()


C= nx.average_clustering(cg)
print("Coeficiente de clustering C para cg: " + str(C))

l=nx.average_shortest_path_length(cg)
print("Camino mínimo medio l para cg: " + str(l))   

#random graph con la misma distribucón de grado    
rg = nx.random_degree_sequence_graph(sorted([e[1] for e in list(nx.degree(cg))], reverse=True))     

Grafo componente gigante
Coeficiente de clustering C para cg: 0.4831015095537764
Camino mínimo medio l para cg: 6.104922041109509


Como el grafo aleatorio presenta nodos que no conectan con nada, al querer calcular el l da error, por lo tanto se va calcular las métricas e la componente gigante del grafo aleatorio

In [8]:
cgrg = max(nx.connected_component_subgraphs(rg), key=len)

print('Componente gigante Random Grafo')
nx.draw(cgrg)  # networkx draw()
plt.draw()  # pyplot draw()
plt.show()

print("Coeficiente de clustering C para cgrg: " + str(nx.average_clustering(cgrg)))    
print("Camino mínimo medio l para cgrg: " + str(nx.average_shortest_path_length(cgrg)))

Componente gigante Random Grafo
Coeficiente de clustering C para rg: 0.003997238649422025
Camino mínimo medio l para rg: 4.74504994720301


Una Red de mundo pequeño es un tipo de grafo para el que la mayoría de los nodos no son vecinos entre sí, y sin embargo la mayoría de los nodos pueden ser alcanzados desde cualquier nodo origen a través de un número relativamente corto de saltos entre ellos.

Al computar los coeficientes de clustering (C) y de camino mínimo medio (l) tanto para la componente gigante del grafo original como para la componente gigante de grafo aleatrorio, encontramos que ambas presentan un valor de l del mismo orden mientras que el C del grafo alatorio es de dos ordenes de magnitud menor. Estos resultados estarían indicando que se trata de un grafo con estructura de mundos pequeños. 

## 6. Estrellas
Discuta cómo haría para individulizar a los autores “estrella” del campo. Evalúe quiénes son estos autores
según por lo menos dos métricas diferentes. Analice qué sucedería con la comunidad si estos autores
desapareciesen. En particular, determine cuántos autores deberían desaparecer para que desaparezca
la componente gigante del grafo.

In [26]:
degree_dict = dict(cg.degree(cg.nodes())) # Run degree centrality
betweenness_dict = nx.betweenness_centrality(cg) # Run betweenness centrality
eigenvector_dict = nx.eigenvector_centrality(cg) # Run eigenvector centrality

# Assign each to an attribute in your network
nx.set_node_attributes(G, betweenness_dict, 'betweenness')
nx.set_node_attributes(G, eigenvector_dict, 'eigenvector')
nx.set_node_attributes(cg, degree_dict, 'degree')

sorted_degree = sorted(degree_dict.items(), key=operator.itemgetter(1), reverse=True)
sorted_betweenness = sorted(betweenness_dict.items(), key=operator.itemgetter(1), reverse=True)
sorted_eigenvector = sorted(eigenvector_dict.items(), key=operator.itemgetter(1), reverse=True)

#top 5 nodes by degree as a list

top_degree = sorted_degree[:5]
print("Top 5 nodos por Degree")
x=[]
y=[]
for td in top_degree:
    x.append(td[0])
    y.append(td[1])
#    print("Name:", td[0], "| Degree Centrality:", td[1])
print(" ")

x1 = np.arange(5)
plt.bar(x1, y,0.5, align='center')
plt.xticks(x1, x)
plt.title('Estrellas según su grado')
plt.xlabel('Autores')
plt.ylabel('Grado')
plt.show()
plt.close()

#top 5 nodes by betweenness as a list
top_betweenness = sorted_betweenness[:5]


print("Top 5 nodos por Betweenness")
x=[]
y=[]
for tb in top_betweenness: # Loop through top_betweenness
    x.append(tb[0])
    y.append(tb[1])
    print("Name:", tb[0], "| Betweenness Centrality:", tb[1])
#    degree = degree_dict[tb[0]] # Use degree_dict to access a node's degree, see footnote 2
#    eigenvector=eigenvector_dict[tb[0]]
#    print("Name:", tb[0], "| Betweenness Centrality:", tb[1], "| Degree:", degree, "| Eigenvector:", eigenvector)

x1 = np.arange(5)
plt.bar(x1, y,0.5, align='center')
plt.xticks(x1, x)
plt.title('Estrellas según su betweenness')
plt.xlabel('Autores')
plt.ylabel('Betweenness')
plt.show()
plt.close()

print(" ")

#top 5 nodes by eigenvector as a list
x=[]
y=[]
top_eigenvector = sorted_eigenvector[:5]
print("Top 5 nodos por Eigenvector")
for te in top_eigenvector: 
    x.append(te[0])
    y.append(te[1])
#    degree = degree_dict[te[0]] 
#    betweenness=betweenness_dict[te[0]]
#    print("Name:", te[0], "| Eigenvector Centrality:", te[1], "| Degree:", degree, "| Betweenness:", eigenvector)
x1 = np.arange(5)
plt.bar(x1, y,0.5, align='center')
plt.xticks(x1, x)
plt.title('Estrellas según su eigenvector')
plt.xlabel('Autores')
plt.ylabel('Eigenvector')
plt.show()
plt.close()

Top 5 nodos por Degree
 
Top 5 nodos por Betweenness
Name: LEE | Betweenness Centrality: 0.07846171774755167
Name: VAFA | Betweenness Centrality: 0.05497143675504384
Name: KIM | Betweenness Centrality: 0.050997965023502255
Name: AMBJORN | Betweenness Centrality: 0.03746351188975207
Name: KOGAN | Betweenness Centrality: 0.036598840419422254
 
Top 5 nodos por Eigenvector


Degree es la forma más común de identificar nodos importantes. Los nodos con mayor degree en nuestro caso, son los autores con mayor cantidad de publicaciones.

Eigenvector es como una extensión de degree, es una suerte de combinación de aristas de ese nodo y aristas de los nodos vecinos. Esta medida en nuestro caso representaria no solo la cantidad de publicaciones de ese nodo sino la cantidad de publicaciones de los nodos vecinos. De alguna forma evalúa qué nodos pueden obtener información de otros nodos más rápido.

Betweenness, por otro lado, mira todos los "caminos más cortos" que pasan por un nodo particular. Es una medida buena para identificar nodos que conectan dos partes de la red que de otra forma estarían desconectadas. Identifica nodos que son importantes no por su cantidad de conexiones, sino por ser puente entre grupos, dándole a la red conectividad y cohesión.




In [28]:
#Primero eliminamos al autor con mayor betweenness
print("Cantidad de nodos componente gigante: " + str(len(list(cg.nodes()))))
print("clustering: " + str(C))
sg1=cg.copy()
sg1.remove_node('LEE')
cgsg1=max(nx.connected_component_subgraphs(sg1), key=len)
C= nx.average_clustering(cgsg1)
cant_nodos = len(list(cgsg1.nodes()))
print("Cantidad de nodos componente sin LEE: " + str(cant_nodos))
print("clustering: " + str(C))

sg2=sg1.copy()
sg2.remove_node('VAFA')
cgsg2=max(nx.connected_component_subgraphs(sg2), key=len)
C= nx.average_clustering(cgsg2)
cant_nodos = len(list(cgsg2.nodes()))
print("Cantidad de nodos componente sin VAFA: " + str(cant_nodos))
print("clustering: " + str(C))

sg3=sg2.copy()
sg3.remove_node('KIM')
cgsg3=max(nx.connected_component_subgraphs(sg3), key=len)
C= nx.average_clustering(cgsg3)
cant_nodos = len(list(cgsg3.nodes()))
print("Cantidad de nodos componente sin KIM: " + str(cant_nodos))
print("clustering: " + str(C))



Cantidad de nodos componente gigante: 3011
clustering: 0.4831015095537764
Cantidad de nodos componente sin LEE: 3006
clustering: 0.48213492998730684
Cantidad de nodos componente sin VAFA: 3005
clustering: 0.4819371497758789
Cantidad de nodos componente sin KIM: 3000
clustering: 0.4821412074016491


In [ ]:
def sacarAutor(autor, grafo):
    grafo.remove_node(autor)
    componente=max(nx.connected_component_subgraphs(grafo), key=len)
    cantidad = len(list(componente.nodes()))
    C= nx.average_clustering(componente)
    print("Cantidad de nodos componente sin VAFA: " + str(cantidad))
    print("clustering: " + str(C))
    return componente
    

In [35]:
grafo = cg.copy()
resultados = []
cant_nodos = len(list(cg.nodes()))
cant_actual = cant_nodos

while cant_actual>cant_nodos/2:
    resultados.append(cant_actual)
    autor = sorted_degree.pop(0)
    
    grafo.remove_node(autor[0])
    componente=max(nx.connected_component_subgraphs(grafo), key=len)
    cant_actual = len(list(componente.nodes()))
    C= nx.average_clustering(componente)
    print("Cantidad de nodos componente sin " +str(autor[0]) +': ' + str(cant_actual))
    print("clustering: " + str(C))
        
        

Cantidad de nodos componente sin LEE: 3006
clustering: 0.48213492998730684
Cantidad de nodos componente sin AMBJORN: 3005
clustering: 0.480625198737054
Cantidad de nodos componente sin PARK: 2998
clustering: 0.48026084272120145
Cantidad de nodos componente sin KIM: 2992
clustering: 0.48061791129178083
Cantidad de nodos componente sin FERRARA: 2991
clustering: 0.4801833987068413
Cantidad de nodos componente sin VAFA: 2990
clustering: 0.4799871436824094
Cantidad de nodos componente sin STROMINGER: 2989
clustering: 0.4798616869469787
Cantidad de nodos componente sin GIBBONS: 2987
clustering: 0.47985173338786546
Cantidad de nodos componente sin ODINTSOV: 2985
clustering: 0.47907925086538855
Cantidad de nodos componente sin POPE: 2984
clustering: 0.4788439050398123
Cantidad de nodos componente sin LU: 2981
clustering: 0.47757946321853506
Cantidad de nodos componente sin DAS: 2978
clustering: 0.47791820701158283
Cantidad de nodos componente sin IVANOV: 2974
clustering: 0.4773521572244077
Can

In [31]:
grafo = cg.copy()
resultados = []
cant_nodos = len(list(cg.nodes()))
cant_actual = cant_nodos

while cant_actual>cant_nodos/2:
    resultados.append(cant_actual)
    autor = sorted_betweenness.pop(0)
    
    grafo.remove_node(autor[0])
    componente=max(nx.connected_component_subgraphs(grafo), key=len)
    cant_actual = len(list(componente.nodes()))
    C= nx.average_clustering(componente)
    print("Cantidad de nodos componente sin " +str(autor[0]) +': ' + str(cant_actual))
    print("clustering: " + str(C))
        
        

Cantidad de nodos componente sin VAFA: 3010
clustering: 0.48290389043616555
Cantidad de nodos componente sin KIM: 3006
clustering: 0.48272441097725843
Cantidad de nodos componente sin AMBJORN: 3005
clustering: 0.4812148758937244
Cantidad de nodos componente sin KOGAN: 3004
clustering: 0.48106494422577156
Cantidad de nodos componente sin PARK: 2999
clustering: 0.48126854859927265
Cantidad de nodos componente sin GIBBONS: 2997
clustering: 0.48124837317167
Cantidad de nodos componente sin OHTA: 2995
clustering: 0.4810938443209473
Cantidad de nodos componente sin FERRARA: 2994
clustering: 0.48067521042943184
Cantidad de nodos componente sin LI: 2993
clustering: 0.48127211942942133
Cantidad de nodos componente sin IVANOV: 2989
clustering: 0.48071745206399086
Cantidad de nodos componente sin SUZUKI: 2978
clustering: 0.4804462650268963
Cantidad de nodos componente sin JOHNSON: 2976
clustering: 0.4805215773471642
Cantidad de nodos componente sin KLEBANOV: 2975
clustering: 0.4803219231125328
Ca

In [36]:
plt.plot(resultados)
plt.show()